In [3]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
#x = x.to_sparse()
data = Data(x=x, edge_index=edge_index.t().contiguous())

In [7]:
from torch_geometric.nn import GCNConv

conv1 = GCNConv(1, 1)

x, edge_index = data.x, data.edge_index

out = conv1(x, edge_index)

In [8]:
out.shape

torch.Size([3, 1])

In [42]:
from typing import List, Optional, Tuple, Union

import torch.nn.functional as F
from torch import Tensor
import torch.nn as nn
from torch.nn import LSTM
from torch_sparse import SparseTensor, matmul

from torch_geometric.nn.aggr import Aggregation, MultiAggregation
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.typing import Adj, OptPairTensor, Size


# TODO
# what is edge attribute

class TestConv(MessagePassing):
   
    def __init__(
        self,
        in_channels: Union[int, Tuple[int, int]],
        out_channels: int,
        aggr: Optional[Union[str, List[str], Aggregation]] = "max",
    ):
        self.in_channels = in_channels
        self.out_channels = out_channels


        super().__init__(aggr)

        # calculate off-set delta x
        self.lin_h = nn.Linear(self.in_channels, self.out_channels)
        # calculate edge update
        self.lin_f = nn.Linear(self.in_channels * 2, self.out_channels)
        # calculate final state update
        self.lin_g = nn.Sequential(
          nn.Linear(self.in_channels, 64),
          nn.ReLU(),
          nn.Linear(64,self.out_channels)
        )
        # self.lin_g = nn.Linear(self.in_channels, self.out_channels)
        #self.reset_parameters()

    def reset_parameters(self):
        self.lin_h.reset_parameters()
        self.lin_f.reset_parameters()
        self.lin_g.reset_parameters()

    def forward(self, x, pos, edge_index):
        out = self.propagate(edge_index, x=x, pos=pos)
        out = self.lin_g(out)
        return x + out

    def message(self, pos_j, pos_i, x_i, x_j):
        delta = self.lin_h(x_i)
        e = torch.cat([pos_j - pos_i + delta, x_j], dim=-1)
        e = self.lin_f(e)
        return e

In [12]:
import sys
sys.path.append('/nfs/homedirs/bindera/reference_implementations/Point-GNN-PyTorch/src/point_gnn_pytorch/')
from models import PointGNN

import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
#x = x.to_sparse()
data = Data(x=x, edge_index=edge_index.t().contiguous())

# conv1 = TestConv(1, 1, aggr="max")

model = PointGNN(1,1)

x, edge_index = data.x, data.edge_index

out = model(x, x, edge_index)
out

AttributeError: 'Sequential' object has no attribute 'reset_parameters'

In [9]:
x.shape

torch.Size([3, 1])

In [17]:
torch.concat((x,x), dim=0).shape

torch.Size([6, 1])

In [20]:
torch.repeat_interleave(x, 2, dim=0).shape

torch.Size([6, 1])

In [24]:
x.shape

torch.Size([3, 1])

Edge Update: https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/conv/gat_conv.html#GATConv


In [27]:
from typing import Tuple, Union

from torch import Tensor
from torch_sparse import SparseTensor, matmul

from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.typing import Adj, OptPairTensor, OptTensor, Size


class GraphConv(MessagePassing):
   
    def __init__(
        self,
        in_channels: Union[int, Tuple[int, int]],
        out_channels: int,
        aggr: str = 'add',
        bias: bool = True,
        **kwargs,
    ):
        super().__init__(aggr=aggr, **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels

        if isinstance(in_channels, int):
            in_channels = (in_channels, in_channels)

        self.lin_rel = Linear(in_channels[0], out_channels, bias=bias)
        self.lin_root = Linear(in_channels[1], out_channels, bias=False)

        self.reset_parameters()

    def reset_parameters(self):
        self.lin_rel.reset_parameters()
        self.lin_root.reset_parameters()


    def forward(self, x: Union[Tensor, OptPairTensor], edge_index: Adj,
                edge_weight: OptTensor = None, size: Size = None) -> Tensor:
        """"""
        if isinstance(x, Tensor):
            x: OptPairTensor = (x, x)

        # propagate_type: (x: OptPairTensor, edge_weight: OptTensor)
        out = self.propagate(edge_index, x=x, edge_weight=edge_weight,
                             size=size)
        #out = self.lin_rel(out)

        print(x[0])
        print(x[1])


        x_r = x[1]
        #if x_r is not None:
        #    out = out + self.lin_root(x_r)

        return out


    def message(self, x_j: Tensor, edge_weight: OptTensor) -> Tensor:
        print(edge_weight)
        return x_j if edge_weight is None else edge_weight.view(-1, 1) * x_j



In [28]:
conv1 = GraphConv(1, 1, aggr="max")

x, edge_index = data.x, data.edge_index

out = conv1(x, edge_index)

None
tensor([[-1.],
        [ 0.],
        [ 1.]])
tensor([[-1.],
        [ 0.],
        [ 1.]])


In [22]:
x

tensor([[-1.],
        [ 0.],
        [ 1.]])

In [18]:
from src.adversarial_attacks_on_feature_dependent_graphs.dataset import PointCloudDataset
from src.adversarial_attacks_on_feature_dependent_graphs.data import *

val_dataset_v2 = PointCloudDataset(split='val')

The size of train data is 9843
Load processed data from ../../../shared/modelnet/modelnet40_normal_resampled/modelnet40_train_1024pts_fps.dat...
The size of test data is 2468
Load processed data from ../../../shared/modelnet/modelnet40_normal_resampled/modelnet40_test_1024pts_fps.dat...


100%|██████████| 1968/1968 [00:45<00:00, 43.04it/s]


In [19]:
val_dataset_v2

PointCloudDataset(1968)

In [20]:
path_to_data = '/nfs/students/bindera/data/modelnet_pyg/val_dataset.pt'
torch.save(val_dataset_v2, path_to_data)

In [3]:
import torch
test_dataset = torch.load(path_to_data)

In [ ]:
sys.path.append('/nfs/homedirs/bindera/reference_implementations/Point-GNN-PyTorch/src/point_gnn_pytorch')

In [19]:
sample = test_dataset[0]

In [46]:
%load_ext autoreload

%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.append('/nfs/homedirs/bindera/reference_implementations/Point-GNN-PyTorch/src/point_gnn_pytorch/')
from models import PointGNN
from layers import PointGNNConv

import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
#x = x.to_sparse()
data = Data(x=x, edge_index=edge_index.t().contiguous())

#  sample = 

# conv1 = TestConv(1, 1, aggr="max")

#sample = [0]
test_dataset
data = test_dataset[0]

data = Data(x=data.x, edge_index=data.edge_index.t().contiguous(), y=data.y)

x, edge_index = data.x, data.edge_index



# conv = PointGNNConv(state_channels = 3,
#         pos_channels = 3,
#         hidden_channels = 64)

# out = conv(x, x, edge_index)

model = PointGNN(num_node_features = 3, 
                    num_classes = 5)
model_out = model(data)
model_out.shape
# out.shape

torch.Size([1, 5])

In [21]:
from torch_geometric.loader import DataLoader
# data = test_dataset[0]

# data = Data(x=data.x, edge_index=data.edge_index.t().contiguous(), y=data.y)
train_loader = DataLoader(val_dataset_v2, batch_size=16, shuffle=True) 
data = next(iter(train_loader))
out = model(data)

In [56]:
data = test_dataset[0]

In [59]:
data

DataBatch(x=[4096, 3], edge_index=[2, 131072], y=[4], batch=[4096], ptr=[5])

In [22]:
out.shape

torch.Size([16, 5])

In [28]:
data

Data(x=[1024, 3], edge_index=[32768, 2], y=[1])

In [27]:
data.edge_index.t().contiguous().shape

torch.Size([2, 32768])

In [29]:
new_data = Data(x=data.x, edge_index=data.edge_index.t().contiguous(), y=data.y)

In [37]:
from torch_geometric.nn import MLP

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

mlp = MLP([1, 32, 64, 128])

mlp(x).shape

torch.Size([3, 128])

In [1]:
import wandb

# wandb.init(project="my-test-project")

In [1]:
import torch
from torch_sparse import SparseTensor

from torch_geometric.nn import ARMAConv, MLP
from torch_geometric.testing import is_full_test

from src.adversarial_attacks_on_feature_dependent_graphs.baselines import PointGNNConv


/nfs/homedirs/bindera/miniconda3/envs/test_torch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
MLP_h = MLP([3, 64, 3])
MLP_h.channel_list

[3, 64, 3]

In [20]:
%load_ext autoreload
%autoreload 2

In [35]:
MLP_h = MLP([3, 64, 3])
MLP_f = MLP([6, 64, 3])
MLP_g = MLP([3, 64, 3])

conv = PointGNNConv(state_channels=3, MLP_h=MLP_h, MLP_f=MLP_f, MLP_g=MLP_g)
assert conv.__repr__() == 'PointGNNConv(3, lin_h=MLP(3, 64, 3), lin_f=MLP(6, 64, 3),lin_g=MLP(3, 64, 3))'
# conv.__repr__()

In [39]:
t = '(Tensor, Tensor, Adj) -> Tensor'
jit = torch.jit.script(conv.jittable(t))
assert jit(x, x, edge_index).tolist() == out.tolist()

RuntimeError: 
Arguments for call are not valid.
The following variants are available:
  
  propagate__0(__torch__.PointGNNConvJittable_f382d4.PointGNNConvJittable_f382d4 self, Tensor edge_index, Tensor x, Tensor pos, (int, int)? size) -> Tensor:
  Expected a value of type 'Tensor' for argument 'edge_index' but instead found type 'Union[Tensor, __torch__.torch_sparse.tensor.SparseTensor]'.
  
  propagate__1(__torch__.PointGNNConvJittable_f382d4.PointGNNConvJittable_f382d4 self, __torch__.torch_sparse.tensor.SparseTensor edge_index, Tensor x, Tensor pos, (int, int)? size) -> Tensor:
  Expected a value of type '__torch__.torch_sparse.tensor.SparseTensor (of Python compilation unit at: 0x5028480)' for argument 'edge_index' but instead found type 'Union[Tensor, __torch__.torch_sparse.tensor.SparseTensor]'.

The original call is:
  File "/tmp/bindera_pyg/tmpd9y4phyc.py", line 238
        # type: (Tensor, Tensor, Adj) -> Tensor
        # propagate_type: (x: Tensor, pos: Tensor)
        out =

In [41]:
from src.adversarial_attacks_on_feature_dependent_graphs.baselines import PointGNNConv
torch.manual_seed(42)
x = torch.randn(6, 3)
edge_index = torch.tensor([[0, 1, 1, 1, 2, 5], [1, 2, 3, 4, 3, 4]])
row, col = edge_index
adj = SparseTensor(row=row, col=col, sparse_sizes=(6, 6))
MLP_h = MLP([3, 64, 3])
MLP_f = MLP([6, 64, 3])
MLP_g = MLP([3, 64, 3])

conv = PointGNNConv(state_channels=3, MLP_h=MLP_h, MLP_f=MLP_f, MLP_g=MLP_g)
assert conv.__repr__() == 'PointGNNConv(3, lin_h=MLP(3, 64, 3), lin_f=MLP(6, 64, 3),lin_g=MLP(3, 64, 3))'
out = conv(x, x, edge_index)

assert out.size() == (6, 3)
assert torch.allclose(conv(x, x, adj.t()), out, atol=1e-6)

if True:
    t = '(Tensor, Tensor, Tensor) -> Tensor'
    jit = torch.jit.script(conv.jittable(t))
    assert jit(x, x, edge_index).tolist() == out.tolist()

    t = '(Tensor, Tensor, SparseTensor) -> Tensor'
    jit = torch.jit.script(conv.jittable(t))
    assert torch.allclose(jit(x, x, adj.t()), out, atol=1e-5)



RuntimeError: 
Arguments for call are not valid.
The following variants are available:
  
  propagate__0(__torch__.PointGNNConvJittable_6f899b.PointGNNConvJittable_6f899b self, Tensor edge_index, Tensor x, Tensor pos, (int, int)? size) -> Tensor:
  Argument size not provided.
  
  propagate__1(__torch__.PointGNNConvJittable_6f899b.PointGNNConvJittable_6f899b self, __torch__.torch_sparse.tensor.SparseTensor edge_index, Tensor x, Tensor pos, (int, int)? size) -> Tensor:
  Expected a value of type '__torch__.torch_sparse.tensor.SparseTensor (of Python compilation unit at: 0x5028480)' for argument 'edge_index' but instead found type 'Tensor'.

The original call is:
  File "/tmp/bindera_pyg/tmp1pwnyawp.py", line 238
        # type: (Tensor, Tensor, Tensor) -> Tensor
        # propagate_type: (x: Tensor, pos: Tensor)
        out = self.propagate(edge_index, x=x, pos=pos)
              ~~~~~~~~~~~~~~ <--- HERE
        out = self.lin_g(out)
        return x + out


In [44]:
from src.adversarial_attacks_on_feature_dependent_graphs.baselines import PointGNNConv
torch.manual_seed(42)
x = torch.randn(6, 3)
edge_index = torch.tensor([[0, 1, 1, 1, 2, 5], [1, 2, 3, 4, 3, 4]])
row, col = edge_index
adj = SparseTensor(row=row, col=col, sparse_sizes=(6, 6))
MLP_h = MLP([3, 64, 3])
MLP_f = MLP([6, 64, 3])
MLP_g = MLP([3, 64, 3])

conv = PointGNNConv(state_channels=3, MLP_h=MLP_h, MLP_f=MLP_f, MLP_g=MLP_g)
assert conv.__repr__() == 'PointGNNConv(3, lin_h=MLP(3, 64, 3),' \
                        ' lin_f=MLP(6, 64, 3),lin_g=MLP(3, 64, 3))'
#string = 'PointGNNConv(3, lin_h=MLP(3, 64, 3), lin_f=MLP(6, 64, 3),lin_g=MLP(3, 64, 3))'
#assert conv.__repr__() == 'PointGNNConv(3, lin_h=MLP(3, 64, 3), lin_f=MLP(6, 64, 3),lin_g=MLP(3, 64, 3))'
out = conv(x, x, edge_index)

In [45]:
float(1e-3)

[autoreload of src.adversarial_attacks_on_feature_dependent_graphs.baselines failed: Traceback (most recent call last):
  File "/nfs/homedirs/bindera/miniconda3/envs/test_torch/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/nfs/homedirs/bindera/miniconda3/envs/test_torch/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/nfs/homedirs/bindera/miniconda3/envs/test_torch/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/nfs/homedirs/bindera/miniconda3/envs/test_torch/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/nfs/homedirs/bin

0.001

In [10]:

t = '(Tensor, Tensor) -> Tensor'
jit = torch.jit.script(conv.jittable(t))
assert jit(x, edge_index).tolist() == out.tolist()

# t = '(Tensor, SparseTensor, OptTensor) -> Tensor'
# jit = torch.jit.script(conv.jittable(t))
# assert torch.allclose(jit(x, adj.t()), out, atol=1e-6)

RuntimeError: 
Unknown type name 'Adj':
  File "/nfs/homedirs/bindera/Adversarial Attacks on Feature-Dependent Graphs/src/adversarial_attacks_on_feature_dependent_graphs/baselines.py", line 90
    def forward(self, x: Tensor, pos: Tensor, edge_index: Adj) -> Tensor:
                                                          ~~~ <--- HERE
        # propagate_type: (x: Tensor, pos: Tensor)
        out = self.propagate(edge_index, x=x, pos=pos)
'PointGNNConvJittable_24da9e.message' is being compiled since it was called from 'PointGNNConvJittable_24da9e.propagate__0'
  File "/tmp/bindera_pyg/tmpyq5ljxes.py", line 217
    
        kwargs = self.__collect__(edge_index, the_size, in_kwargs)
        out = self.message(pos_i=kwargs.pos_i, pos_j=kwargs.pos_j, x_i=kwargs.x_i, x_j=kwargs.x_j)
        ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
        out = self.aggregate(out, ptr=kwargs.ptr, dim_size=kwargs.dim_size, index=kwargs.index)
     

In [19]:
assert conv(x, x, adj.t()).tolist() == out.tolist()

AssertionError: 

In [5]:
import torch

def test_sum(a, b):
    return a + b

# Annotate the arguments to be int
scripted_fn = torch.jit.script(test_sum, example_inputs=[(3, 4)])

print(type(scripted_fn))  # torch.jit.ScriptFunction

# See the compiled graph as Python code
print(scripted_fn.code)

# Call the function using the TorchScript interpreter
scripted_fn(20, 100)

<class 'torch.jit.ScriptFunction'>
def test_sum(a: Tensor,
    b: Tensor) -> Tensor:
  return torch.add(a, b)



/nfs/homedirs/bindera/miniconda3/envs/test_torch/lib/python3.7/site-packages/torch/jit/_script.py:1280: UserWarning: Warning: monkeytype is not installed. Please install https://github.com/Instagram/MonkeyType to enable Profile-Directed Typing in TorchScript. Refer to https://github.com/Instagram/MonkeyType/blob/master/README.rst to install MonkeyType. 
  warnings.warn("Warning: monkeytype is not installed. Please install https://github.com/Instagram/MonkeyType "


RuntimeError: test_sum() Expected a value of type 'Tensor (inferred)' for argument 'a' but instead found type 'int'.
Inferred 'a' to be of type 'Tensor' because it was not annotated with an explicit type.
Position: 0
Value: 20
Declaration: test_sum(Tensor a, Tensor b) -> Tensor
Cast error details: Unable to cast 20 to Tensor

In [20]:
conv(x, x, adj.t()).tolist()

[[0.7913013100624084, 0.038601651787757874, -0.1411139965057373],
 [-0.7865033745765686, 0.4582751393318176, -0.742428719997406],
 [1.0047038793563843, -1.337821125984192, 1.5362837314605713],
 [-0.29695242643356323, -0.15038949251174927, 0.4561692774295807],
 [0.010547250509262085, 0.21245476603507996, 0.4234809875488281],
 [0.7835968136787415, 0.7526127099990845, 1.9474248886108398]]

In [12]:
conv(x, x, adj.t()).tolist()

[[2.5826876163482666, -0.30090582370758057, -0.5688638091087341],
 [-0.5629196763038635, 0.1901358813047409, -1.791551113128662],
 [-1.0719306468963623, -0.5419655442237854, 2.1522042751312256],
 [-2.713207721710205, -2.304295778274536, 0.11807659268379211],
 [-0.32690030336380005, -0.8785462975502014, -0.6353766918182373],
 [2.14768385887146, 0.22164282202720642, 1.0005592107772827]]

In [13]:
out.tolist()

[[2.5826878547668457, -0.30090591311454773, -0.5688638091087341],
 [-0.5629197955131531, 0.19013582170009613, -1.791551113128662],
 [-1.0719306468963623, -0.5419654846191406, 2.1522042751312256],
 [-2.713207721710205, -2.3042960166931152, 0.1180766373872757],
 [-0.3269002437591553, -0.8785462975502014, -0.6353767514228821],
 [2.14768385887146, 0.22164276242256165, 1.0005593299865723]]

In [ ]:
def test_pointgnn_conv():
    x = torch.randn(6, 3)
    edge_index = torch.tensor([[0, 1, 1, 1, 2, 5], [1, 2, 3, 4, 3, 4]])
    row, col = edge_index
    adj = SparseTensor(row=row, col=col, sparse_sizes=(6, 6))

    conv = PointGNNConv(16, 32, num_stacks=8, num_layers=4)
    # assert conv.__repr__() == 'ARMAConv(16, 32, num_stacks=8, num_layers=4)'
    out = conv(x, edge_index)
    assert out.size() == (4, 32)
    assert conv(x, adj.t()).tolist() == out.tolist()

    if is_full_test():
        t = '(Tensor, Tensor, OptTensor) -> Tensor'
        jit = torch.jit.script(conv.jittable(t))
        assert jit(x, edge_index).tolist() == out.tolist()

        t = '(Tensor, SparseTensor, OptTensor) -> Tensor'
        jit = torch.jit.script(conv.jittable(t))
        assert torch.allclose(jit(x, adj.t()), out, atol=1e-6)


In [15]:
x = torch.randn(4, 16)
edge_index = torch.tensor([[0, 0, 0, 1, 2, 3], [1, 2, 3, 0, 0, 0]])
row, col = edge_index
adj = SparseTensor(row=row, col=col, sparse_sizes=(4, 4))

conv = ARMAConv(16, 32, num_stacks=8, num_layers=4)
assert conv.__repr__() == 'ARMAConv(16, 32, num_stacks=8, num_layers=4)'
out = conv(x, edge_index)
assert out.size() == (4, 32)
assert conv(x, adj.t()).tolist() == out.tolist()

In [6]:
import torch

@torch.jit.script
def foo(x, y):
    if x.max() > y.max():
        r = x
    else:
        r = y
    return r

print(type(foo))  # torch.jit.ScriptFunction

# See the compiled graph as Python code
print(foo.code)

# Call the function using the TorchScript interpreter
foo(torch.ones(2, 2), torch.ones(2, 2))

<class 'torch.jit.ScriptFunction'>
def foo(x: Tensor,
    y: Tensor) -> Tensor:
  _0 = bool(torch.gt(torch.max(x), torch.max(y)))
  if _0:
    r = x
  else:
    r = y
  return r



tensor([[1., 1.],
        [1., 1.]])

In [12]:
from torch_geometric.nn import ClusterGCNConv

x = torch.randn(4, 16)
edge_index = torch.tensor([[0, 1, 2, 3], [0, 0, 1, 1]])
row, col = edge_index
adj = SparseTensor(row=row, col=col, sparse_sizes=(4, 4))

conv = ClusterGCNConv(16, 32, diag_lambda=1.)
assert conv.__repr__() == 'ClusterGCNConv(16, 32, diag_lambda=1.0)'
out = conv(x, edge_index)
assert out.size() == (4, 32)
assert torch.allclose(conv(x, adj.t()), out, atol=1e-5)

if True:
    t = '(Tensor, Tensor) -> Tensor'
    jit = torch.jit.script(conv.jittable(t))
    assert jit(x, edge_index).tolist() == out.tolist()

In [13]:
jit

RecursiveScriptModule(
  original_name=ClusterGCNConvJittable_fd4d57
  (aggr_module): RecursiveScriptModule(original_name=SumAggregation)
  (lin_out): RecursiveScriptModule(original_name=Linear)
  (lin_root): RecursiveScriptModule(original_name=Linear)
)